In [1]:
!pip install keras-tuner -q

import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
import keras_tuner
from google.colab import drive
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.optimizers import AdamW

drive.mount('/content/drive')
print("TensorFlow Version:", tf.__version__)
print("KerasTuner Version:", keras_tuner.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.2 MB/s eta 0:00:00
Mounted at /content/drive
TensorFlow Version: 2.19.0
KerasTuner Version: 1.4.7


In [2]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
CLASSES = ["acne", "pigmentation", "wrinkles"]
DATA_ROOT = "/content/drive/MyDrive/skincareapp/acne clean pigmentation wrinkles/"

df = pd.read_csv(os.path.join(DATA_ROOT, "labels.csv"))
df["filename"] = df["filename"].apply(lambda x: os.path.join(DATA_ROOT, x))

In [3]:
train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=42, stratify=df[CLASSES])
train_df, val_df = train_test_split(train_val_df, test_size=0.15, random_state=42, stratify=train_val_df[CLASSES])

pos_counts = train_df[CLASSES].sum().values
total_train_samples = len(train_df)

In [4]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
], name="data_augmentation")

#Create tf.data Pipelines
def parse_function(filename, labels):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image_decoded, tf.float32)
    image_resized = tf.image.resize(image, IMG_SIZE)
    return image_resized, labels

def create_dataset(df, batch_size, augment=False, cache_file=None):
    dataset = tf.data.Dataset.from_tensor_slices(
        (df["filename"].values, df[CLASSES].values.astype(np.float32))
    )
    dataset = dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)

    if augment:
        dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y),
                              num_parallel_calls=tf.data.AUTOTUNE)

    if cache_file:
        dataset = dataset.cache(cache_file)
    else:
        dataset = dataset.cache()
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

train_cache_file = os.path.join(DATA_ROOT, 'resnet_train_cache')
val_cache_file = os.path.join(DATA_ROOT, 'resnet_val_cache')

#Create the datasets
train_ds = create_dataset(train_df, BATCH_SIZE, augment=True, cache_file=train_cache_file)
val_ds = create_dataset(val_df, BATCH_SIZE, augment=False, cache_file=val_cache_file)
test_ds = create_dataset(test_df, BATCH_SIZE, augment=False)

print("tf.data pipelines created successfully.")

tf.data pipelines created successfully.


In [5]:
def build_hyper_model(hp):
    #Defining Hyperparameter for Dropout
    hp_dropout = hp.Float('dropout', 0.2, 0.5, step=0.1)

    #Load the ResNet50V2 base model
    base_model = ResNet50V2(
        include_top=False,
        input_shape=IMG_SIZE + (3,),
        weights="imagenet"
    )

    inputs = layers.Input(shape=IMG_SIZE + (3,))
    x = tf.keras.applications.resnet_v2.preprocess_input(inputs)
    x = base_model(x, training=base_model.trainable) #training=False in stage 1
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(hp_dropout)(x) #Using tunable dropout
    outputs = layers.Dense(len(CLASSES), activation="sigmoid")(x)
    model = Model(inputs, outputs, name="resnet50v2_hyper_model")
    return model

tf.keras.backend.clear_session()

In [6]:
#Serializable Weighted BCE Loss
def create_weighted_bce_loss(pos_counts, total_samples, smooth=0.05):

    #A factory function that creates a weighted BCE loss function.
    pos = tf.constant(pos_counts, dtype=tf.float32)
    neg = total_samples - pos
    w_pos = neg / tf.maximum(pos, 1.0)
    w_neg = tf.ones_like(pos)

    def weighted_bce(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        y_true = y_true * (1.0 - smooth) + 0.5 * smooth
        bce = tf.keras.backend.binary_crossentropy(y_true, y_pred)
        weights = y_true * w_pos + (1.0 - y_true) * w_neg
        return tf.reduce_mean(bce * weights)

    return weighted_bce

#Creating the loss function instance and custom_objects dict
loss_fn = create_weighted_bce_loss(pos_counts, total_train_samples)

#Custom Objects needed for loading/saving
custom_objects = {
    "weighted_bce": loss_fn
}

In [9]:
class CustomTuner(keras_tuner.RandomSearch):

    #Custom Tuner for two-stage training.
    def __init__(self, loss_function, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_function = loss_function

    def run_trial(self, trial, train_ds, val_ds, **kwargs):
        hp = trial.hyperparameters
        model = self.hypermodel.build(hp)

        all_metrics = [
            tf.keras.metrics.BinaryAccuracy(name="acc", threshold=0.5),
            tf.keras.metrics.AUC(name="auc", multi_label=True),
            tf.keras.metrics.Precision(name="precision", thresholds=0.5),
            tf.keras.metrics.Recall(name="recall", thresholds=0.5),
        ]

        #STAGE 1: Train the Head
        print(f"\n[Trial {trial.trial_id}] Stage 1: Training head...")
        head_lr = hp.Float('head_lr', 1e-4, 1e-3, sampling='log')

        #Layer name must match your model
        model.get_layer("resnet50v2").trainable = False

        model.compile(
            optimizer=AdamW(learning_rate=head_lr, weight_decay=1e-4),
            loss=self.loss_function,
            metrics=all_metrics
        )

        model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=10,
            verbose=1
        )

        #STAGE 2: Fine-Tuning
        print(f"\n[Trial {trial.trial_id}] Stage 2: Fine-tuning...")
        finetune_lr = hp.Float('finetune_lr', 1e-6, 5e-5, sampling='log')

        #Layer name must match your model
        model.get_layer("resnet50v2").trainable = True

        model.compile(
            optimizer=AdamW(learning_rate=finetune_lr, weight_decay=1e-4),
            loss=self.loss_function,
            metrics=all_metrics
        )

        callbacks = [
            tf.keras.callbacks.EarlyStopping(
                monitor="val_auc",
                mode="max",
                patience=5,
                restore_best_weights=True
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor="val_auc",
                mode="max",
                factor=0.2,
                patience=5
            )
        ]

        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=40,
            callbacks=callbacks,
            initial_epoch=10,
            verbose=1
        )

        print(f"[Trial {trial.trial_id}] Evaluating best weights on val_ds...")
        eval_results = model.evaluate(val_ds, return_dict=True, verbose=0)

        val_results_with_prefix = {f"val_{k}": v for k, v in eval_results.items()}
        return val_results_with_prefix

In [10]:
#Defining the tuner
tuner = CustomTuner(
    loss_function=loss_fn,
    hypermodel=build_hyper_model,
    objective=keras_tuner.Objective("val_auc", direction="max"),

    max_trials=5,

    executions_per_trial=1,
    directory=os.path.join(DATA_ROOT, 'keras_tuner'),
    project_name='resnet_skin_tuning',
    overwrite=True
)

#Printing a summary of the search space
tuner.search_space_summary()

#Starting the search
print("\nStarting hyperparameter search...")
start_time = time.time()

tuner.search(
    train_ds=train_ds,
    val_ds=val_ds
)

end_time = time.time()
print(f"\nTotal search time: {(end_time - start_time) / 60:.2f} minutes")

Trial 5 Complete [00h 12m 04s]
val_auc: 0.9870684742927551

Best val_auc So Far: 0.9870684742927551
Total elapsed time: 00h 59m 56s

Total search time: 59.93 minutes


In [11]:
#Getting the Best Hyperparameters
print("\nTop Trials")
tuner.results_summary(num_trials=5)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("\nBest Hyperparameters Found")
print(f"Dropout: {best_hps.get('dropout'):.3f}")
print(f"Head LR: {best_hps.get('head_lr'):.1e}")
print(f"Finetune LR: {best_hps.get('finetune_lr'):.1e}")

#Building the FINAL Best Model
print("\nBuilding the best model for final training...")
final_model = build_hyper_model(best_hps)
final_model.summary()

#Defining Callbacks for FINAL Training
FINAL_MODEL_PATH = os.path.join(DATA_ROOT, "resnet_skin_model_FINAL_TUNED.keras")
final_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_auc", mode="max", patience=7, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_auc", mode="max", factor=0.2, patience=3, min_lr=1e-6),
    tf.keras.callbacks.ModelCheckpoint(FINAL_MODEL_PATH, monitor="val_auc", mode="max", save_best_only=True)
]

#STAGE 1: Train the Head (Full Epochs)
print("\nFinal Training: STAGE 1 (Head)")
final_head_lr = best_hps.get('head_lr')

final_model.get_layer("resnet50v2").trainable = False
final_model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=final_head_lr, weight_decay=1e-4),
    loss=loss_fn,
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name="acc", threshold=0.5),
        tf.keras.metrics.AUC(name="auc", multi_label=True),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall")
    ]
)
start_time_stage1 = time.time()
history_head = final_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    verbose=1
)
end_time_stage1 = time.time()

#STAGE 2: Fine-Tuning (Full Epochs)
print("\nFinal Training: STAGE 2 (Fine-Tune)")
final_finetune_lr = best_hps.get('finetune_lr')

final_model.get_layer("resnet50v2").trainable = True
final_model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=final_finetune_lr, weight_decay=1e-4),
    loss=loss_fn,
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name="acc", threshold=0.5),
        tf.keras.metrics.AUC(name="auc", multi_label=True),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall")
    ]
)
start_time_stage2 = time.time()
history_fine_tune = final_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=70,
    callbacks=final_callbacks,
    initial_epoch=len(history_head.history['loss']),
    verbose=1
)
end_time_stage2 = time.time()


Top Trials
Results summary
Results in /content/drive/MyDrive/skincareapp/acne clean pigmentation wrinkles/keras_tuner/resnet_skin_tuning
Showing 5 best trials
Objective(name="val_auc", direction="max")

Trial 4 summary
Hyperparameters:
dropout: 0.2
head_lr: 0.0002667602575502906
finetune_lr: 2.4331543558259883e-05
Score: 0.9870684742927551

Trial 2 summary
Hyperparameters:
dropout: 0.2
head_lr: 0.00026479831891630407
finetune_lr: 7.699317528409584e-06
Score: 0.9864333271980286

Trial 3 summary
Hyperparameters:
dropout: 0.2
head_lr: 0.0007959937530049706
finetune_lr: 2.3715642364138958e-05
Score: 0.9831721186637878

Trial 1 summary
Hyperparameters:
dropout: 0.4
head_lr: 0.0007151963634216637
finetune_lr: 1e-06
Score: 0.9620389938354492

Trial 0 summary
Hyperparameters:
dropout: 0.30000000000000004
head_lr: 0.0001
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_

Model: "resnet50v2_hyper_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide_6 (TrueDivide)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract_6 (Subtract)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50v2 (Functional)         │ (None, 7, 7, 2048)     │    23,564,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_6      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 3)              │         6,147 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,570,947 (89.92 MB)

 Trainable params: 23,525,507 (89.74 MB)

 Non-trainable params: 45,440 (177.50 KB)


Final Training: STAGE 1 (Head)
Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 38s 218ms/step - acc: 0.6886 - auc: 0.5917 - loss: 1.1525 - precision: 0.2402 - recall: 0.2826 - val_acc: 0.8318 - val_auc: 0.8562 - val_loss: 1.0730 - val_precision: 0.5640 - val_recall: 0.6058
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 12s 100ms/step - acc: 0.7658 - auc: 0.7723 - loss: 1.0591 - precision: 0.4287 - recall: 0.6272 - val_acc: 0.8313 - val_auc: 0.8730 - val_loss: 1.0191 - val_precision: 0.5495 - val_recall: 0.7487
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 86ms/step - acc: 0.7818 - auc: 0.8243 - loss: 1.0038 - precision: 0.4606 - recall: 0.7373 - val_acc: 0.8308 - val_auc: 0.8785 - val_loss: 0.9789 - val_precision: 0.5470 - val_recall: 0.7698
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 11s 93ms/step - acc: 0.7918 - auc: 0.8480 - loss: 0.9631 - precision: 0.4770 - recall: 0.7760 - val_acc: 0.8323 - val_auc: 0.8820 - val_loss: 0.9482 - val_precision: 0.5477 - val_recall: 0.8042
Epoch 5/10
115/115 ━━━━━━━━━━━

In [12]:
#Final Evaluation on Test Set
print(f"\nLoading best saved final model from: {FINAL_MODEL_PATH}")
loaded_best_model = tf.keras.models.load_model(
    FINAL_MODEL_PATH,
    custom_objects=custom_objects
)

print("\nEvaluating the final tuned ResNet model on the test set...")
test_results = loaded_best_model.evaluate(test_ds, return_dict=True)

print("\nFinal ResNet Test Set Evaluation Results")
precision = 0.0
recall = 0.0
for metric, value in test_results.items():
    print(f"{metric}: {value:.4f}")
    if metric == "precision":
        precision = value
    if metric == "recall":
        recall = value

if precision + recall > 0:
    f1_score = 2 * (precision * recall) / (precision + recall)
    print(f"f1_score (calculated): {f1_score:.4f}")
else:
    print("f1_score (calculated): 0.0")

#Print Stats
print("\nFinal Model Stats")
if os.path.exists(FINAL_MODEL_PATH):
    file_size_mb = os.path.getsize(FINAL_MODEL_PATH) / (1024 * 1024)
    print(f"Model Size on Disk: {file_size_mb:.2f} MB")

total_time_sec = (end_time_stage1 - start_time_stage1) + (end_time_stage2 - start_time_stage2)
total_epochs_ran = len(history_head.history['loss']) + len(history_fine_tune.history['loss'])
avg_time_per_epoch_sec = total_time_sec / total_epochs_ran
print(f"Total Training Time: {total_time_sec / 60:.2f} minutes")
print(f"Total Epochs Trained: {total_epochs_ran}")


Loading best saved final model from: /content/drive/MyDrive/skincareapp/acne clean pigmentation wrinkles/resnet_skin_model_FINAL_TUNED.keras

Evaluating the final tuned ResNet model on the test set...
24/24 ━━━━━━━━━━━━━━━━━━━━ 21s 453ms/step - acc: 0.9778 - auc: 0.9940 - loss: 0.3217 - precision: 0.9416 - recall: 0.9406

Final ResNet Test Set Evaluation Results
acc: 0.9768
auc: 0.9925
loss: 0.3556
precision: 0.9395
recall: 0.9416
f1_score (calculated): 0.9405

Final Model Stats
Model Size on Disk: 270.18 MB
Total Training Time: 29.85 minutes
Total Epochs Trained: 54
